# Engineering WindFarmModels

All Wind farms models take a `Site` and a `WindTurbines` object as input

In [ ]:
# Install PyWake if needed
try:
    import py_wake
except ModuleNotFoundError:
    !pip install git+https://gitlab.windenergy.dtu.dk/TOPFARM/PyWake.git

In [ ]:
# import and setup site and windTurbines
import numpy as np
import matplotlib.pyplot as plt
import os
import py_wake
from py_wake.examples.data.hornsrev1 import V80, Hornsrev1Site

site = Hornsrev1Site()
windTurbines = V80()
wt_x, wt_y = site.initial_position.T

## Model overview
![Engineering models](../_static/EngineeringModels.svg)

The engineering wind farms models in PyWake are composed of one of two wind farms models in combination with a wake deficit model, a superposition model and optionally a blockage deficit and a turbulence model.

- `WindFarmModel`: Defines the proceedure that detemines how wake and blockage deficits propagates in the wind farm. 
Two models are available:
  - [PropagateDownwind](#PropagateDownwind) (fast, but blockage is neglected)
  - [All2AllIterative](#All2AllIterative) (slower but supports blockage).
- Wake `DeficitModel`: Calculate wake deficit from one wind turbine to downstream wind turbines or sites in the wind farm. Several common models are available: 
  - [NOJDeficit](#NOJDeficit)
  - [FugaDeficit](#FugaDeficit)
  - [BastankhahGaussianDeficit](#BastankhahGaussianDeficit)
  - [NiayifarGaussianDeficit](#NiayifarGaussianDeficit)
  - [ZongGaussianDeficit](#ZongGaussianDeficit)
  - [IEA37SimpleBastankhahGaussianDeficit](#IEA37SimpleBastankhahGaussianDeficit)
  - [GCLDeficit](#GCLDEficit)
- `SuperpositionModel`: Defines how deficits from multiple sources sums up. Available models are:
  - [LinearSum](#LinearSum): Deficits sum up linearly
  - [SquaredSum](#SquaredSum): Deficits sum as root-sum-square
  - [MaxSum](#MaxSum): Only the largest deficit is considered
- Blockage `DeficitModel`: Calculate blockage deficit from one wind turbine to other wind turbines or sites in the wind farm. Some models, model upstream effects only while other also models downstream speed-up effects. Available models are:
  - [SelfSimilarityDeficit](#SelfSimilarityDeficit)
  - [SelfSimilarityDeficit2020](#SelfSimilarityDeficit2020)
  - [FugaDeficit](#FugaDeficit)
  - [VortexCylinder](#VortexCylinder)
  - [VortexDipole](#VortexDipole)
  - [RankineHalfBody](#RankineHalfBody)
  - [HybridInduction](#HybridInduction)
  - [Rathmann](#Rathmann)  
- `RotorAvgModel`: Defines one or more points at the rotor to calculate a (weighted) rotor-average deficit from
  - [RotorCenter](#RotorCenter): One point at the center of the rotor
  - [GridRotorAvg](#GridRotorAvg): Custom grid in Cartesian coordinates
  - [EqGridRotorAvg](#EqGridRotorAvg): Equidistant N x N Cartesian grid covering the rotor
  - [GQGridRotorAvg](#GQGridRotorAvg): M x N cartesian grid using Gaussian quadrature coordinates and weights
  - [PolarGridRotorAvg](#PolarGridRotorAvg): Custom grid in polar coordinates  
  - [CGIRotorAVG](#CGIRotorAvg): Circular Gauss Integration
- `DeflectionModel`: Calculate deflected downwind and crosswind distances due to yaw misalignment, shear etc. Available models are:
  - [JimenezWakeDeflection](#JimenezWakeDeflection)
- `TurbulenceModel`: Calculate added turbulence in the wake from one wind turbine to downstream wind turbines or sites in the wind farm. Available models are:
  - [STF2005TurbulenceModel](#STF2005TurbulenceModel): Steen Frandsen, from IEC 2005 standard
  - [STF2017TurbulenceModel](#STF2017TurbulenceModel): Steen Frandsen, from IEC 2017 standard
  - [GCLTurbulence](#GCLTurbulence): Gunner Chr. Larsen
  - [CrespoHernandez](#CrespoHernandez): A. Crespo and J. Hernández
- `GroundModel`: Model effects of ground:
  - [Mirror](#Mirror): The ground acts as a mirror on the wake


## Predefined WindFarmModels
The deficit models comprise:

| Name | WindFarmModel | Wake DeficitModel | Blockage DeficitModel | SuperpositionModel |
| :--- | :--- | :--- | :--- | :--- | 
| NOJ | [PropagateDownwind](#PropagateDownwind) | [NOJDeficit](#NOJDeficit) | - | [SquaredSum](#SquaredSum) |
| Fuga | [PropagateDownwind](#PropagateDownwind) | [FugaDeficit](#FugaDeficit) | - | [LinearSum](#LinearSum) |
| FugaBlockage | [All2AllIterative](#All2AllIterative) | [FugaDeficit](#FugaDeficit) | [FugaDeficit](#FugaDeficit) | [LinearSum](#LinearSum) |
| BastankhahGaussian | [PropagateDownwind](#PropagateDownwind) | [BastankhahGaussianDeficit](#BastankhahGaussianDeficit) | - | [SquaredSum](#SquaredSum) |
| IEA37SimpleBastankhahGaussian | [PropagateDownwind](#PropagateDownwind) | [IEA37SimpleBastankhahGaussianDeficit](#IEA37SimpleBastankhahGaussianDeficit) | - | [SquaredSum](#SquaredSum) |

- Default rotor-average model: `RotorCenter`
- Default turbulence model: `None`

In [ ]:
from py_wake import NOJ
from py_wake import Fuga
from py_wake import FugaBlockage
from py_wake import BastankhahGaussian
from py_wake import IEA37SimpleBastankhahGaussian

# Path to Fuga look-up tables
lut_path = os.path.dirname(py_wake.__file__)+'/tests/test_files/fuga/2MW/Z0=0.03000000Zi=00401Zeta0=0.00E+00/'

models = {'NOJ': NOJ(site,windTurbines), 
          'Fuga': Fuga(lut_path,site,windTurbines),
          'FugaBlockage': FugaBlockage(lut_path,site,windTurbines), 
          'BGaus': BastankhahGaussian(site,windTurbines),
          'IEA37BGaus': IEA37SimpleBastankhahGaussian(site,windTurbines)
         }

These models can easily be combined with other models, e.g. NOJ with linear sum superposition:

In [ ]:
from py_wake.superposition_models import LinearSum
models['NOJLinear'] = NOJ(site,windTurbines,superpositionModel=LinearSum())

or models can be combined in custom ways, e.g. NOJDeficit for the wake, LinearSum superposition and SelfSimilarityDeficit for the blockage:

In [ ]:
from py_wake.wind_farm_models import All2AllIterative
from py_wake.deficit_models import NOJDeficit, SelfSimilarityDeficit
models['NOJ_ss'] = All2AllIterative(site,windTurbines,
                                          wake_deficitModel=NOJDeficit(),
                                          superpositionModel=LinearSum(), 
                                          blockage_deficitModel=SelfSimilarityDeficit() )

In [ ]:
for name, model in models.items():
    print ("%s: %s"%(name, model))

## Engineering WindFarmModel base classes

### PropagateDownwind
The `PropagateDownwind` wind farm model is very fast as it only performs a minimum of deficit calculations. It iterates over all turbines in downstream order. In each iteration it calculates the effective wind speed at the current wind turbine as the free stream wind speed minus the sum up the deficit from upstream sources. Based on this effective wind speed, it calculates the deficit caused by the current turbine on all downstream destinations. Note, that this procedure neglects upstream blockage effects.

```python

for wt in wind turbines in downstream order:
    ws_eff[wt] = ws[wt] - superposition(deficit[from_upstream_src,to_wt])
    ct = windTurbines.ct(ws_eff[wt])
    deficit[from_wt,to_downstream_dst] = wakeDeficitModel(ct, distances[from_wt,to_downstream_dst], ...)
```

The proceedure is illustrated in the animation below:
- Iteration 1: WT0 sees the free wind (10m/s). Its deficit on WT1 and WT2 is calculated.
- Iteration 2: WT1 sees the free wind minus the deficit from WT0. Its deficit on WT2 is calculated and the effective wind speed at WT2 is updated

![PropagateDownwind](../_static/PropagateDownwind.gif)

In [ ]:
%%timeit -r1
# simulate with 20 wind turbines, 360 wind directions and 23 wind speeds
models['Fuga'](wt_x[:20],wt_y[:20])

### All2AllIterative
The `All2AllIterative` wind farm model is slower but is capable of handling blockage effects. <br/> 
It iterates until the effective wind speed converge (i.e. less than or equal to the maximum number of turbines that affect each other in the wind farm. The converge tolerance is an input parameter).<br/> 
In each iteration it sums up the deficit from all wind turbine sources and calculates the deficit caused by the all wind turbines turbine on all wind turbines.

```python

while ws_eff not converged:
    ws_eff[all] = ws[all] - superposition(deficit[from_all,to_all])
    ct[all] = windTurbines.ct(ws_eff[all])
    deficit[from_all,to_all] = wakeDeficitModel(ct[all], distances[from_all,to_all], ...)
```

The proceedure is illustrated in the animation below:
- Iteration 1: All three WT see the free wind (10m/s) and their CT values and resulting deficits are therefore equal
- Iteration 2: The local effective wind speeds are updated taking into account the wake and blockage effects of the other WT. Based on these wind speeds the CT and deficits are recalculated
- Iteration 3: Repeat after which the flow field has converged

![All2AllIterative](../_static/All2AllIterative.gif)

In [ ]:
%%timeit -r1
# simulate with 20 wind turbines, 360 wind directions and 23 wind speeds
models['FugaBlockage'](wt_x[:20],wt_y[:20])

## Wake deficit models
The wake deficit models compute the wake deficit caused by a single wind turbine. 

**Variable suffixes**

The implementation of the deficit models is highly vectorized and therefore suffixes are used to indicate the dimension of variables. The suffixes used in this context are:

- i: source wind turbines
- j: destination wind turbines
- k: wind speeds
- l: wind directions

This means that `deficit_ijlk[0,1,2,3]` holds the deficit caused by the first turbine on the second turbine for the third  wind direction and fourth wind speed

In [ ]:
# methods to plot deficit map, used below to visualize and compare the deficit models

import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
from py_wake.deficit_models import BastankhahGaussianDeficit
D = 80
def _map(f, xy=None):
    X, Y = np.meshgrid(*xy)
    x_j, y_j = X.flatten(), Y.flatten()
    downwind_distance_ijlk = x_j.reshape((1, -1, 1, 1))
    crosswind_distance_ijlk = np.abs(y_j.reshape((1, -1, 1, 1)))
    ws = 10
    res_ijlk = f(
        # specify arguments for all models
        WS_ilk=np.array([[[ws]]]),  # wind speed at source turbine
        D_src_il=np.array([[D]]),  # diameter of source turbine
        D_dst_ijl=None,  # diameter of destination turbine
        h_il=np.array([[67]]),  # source turbine hub height
        dw_ijlk=downwind_distance_ijlk,  # down wind distance
        hcw_ijlk=crosswind_distance_ijlk,  # horizontal cross wind distance
        dh_ijlk=np.zeros_like(crosswind_distance_ijlk),
        # height difference(vertical cross wind distance) between source and destination turbine
        cw_ijlk=crosswind_distance_ijlk,  # cross wind distance (both horizontal and vertical)
        ct_ilk=np.array([[[8 / 9]]]),  # thrust coefficient
        WS_eff_ilk=np.array([[[ws]]]),  # effective wind speed at source turbine
        TI_ilk=np.array([[[0.1]]]),
        TI_eff_ilk=np.array([[[0.1]]]),
        wake_radius_ijlk=BastankhahGaussianDeficit().wake_radius(dw_ijlk=downwind_distance_ijlk, 
                                                  D_src_il=np.array([[D]]), 
                                                  ct_ilk=np.array([[[8 / 9]]])),
    )
    return X / D, Y / D, res_ijlk[0, :, 0, 0].reshape(X.shape)

    
def plot_deficit_map(model, cmap='Blues', levels=np.linspace(0,10,55)):
    xy = np.linspace(-200,500,200), np.linspace(-200,200,200)
    X,Y,deficit = _map(model.calc_deficit, xy)
    c = plt.contourf(X,Y,deficit, levels=levels, cmap=cmap)
    plt.colorbar(c, label="Deficit [m/s]")
    plt.plot([0,0],[-1/2,1/2],'k')
    plt.ylabel("Crosswind distance [y/D]")
    plt.xlabel("downwind distance [x/D]")

def plot_wake_deficit_map(model):
    cmap = np.r_[[[1,1,1,1],[1,1,1,1]],cm.Blues(np.linspace(-0,1,128))] # ensure zero deficit is white
    plot_deficit_map(model,cmap=ListedColormap(cmap))

def plot_blockage_deficit_map(model):
    from matplotlib import cm
    from matplotlib.colors import ListedColormap, LinearSegmentedColormap
    cmap = np.r_[cm.Reds_r(np.linspace(-0,1,127)),[[1,1,1,1],[1,1,1,1]],cm.Blues(np.linspace(-0,1,128))] # ensure zero deficit is white
    plot_deficit_map(model,cmap=ListedColormap(cmap), levels=np.linspace(-3.5,3.5,113))


### NOJDeficit

The NOJDeficit model is implemented according to Niels Otto Jensen, "A note on wind generator interaction." (1983), i.e. a top-hat wake, only valid in the far wake


In [ ]:
from py_wake.deficit_models import NOJDeficit
plot_wake_deficit_map(NOJDeficit())

### FugaDeficit

The FugaDeficit model calculates the wake deficit based on a set op look-up tables computed by a linearized RANS solver. The look-up tables be created in advance using the [Fuga GUI](https://orbit.dtu.dk/en/publications/developments-of-the-offshore-wind-turbine-wake-model-fuga)

The fugaDeficit models both near wake, far wake and blockage deficit

In [ ]:
import os
import py_wake
from py_wake.deficit_models import FugaDeficit

# Path to Fuga look-up tables
lut_path = os.path.dirname(py_wake.__file__)+'/tests/test_files/fuga/2MW/Z0=0.03000000Zi=00401Zeta0=0.00E+00/'

plot_wake_deficit_map(FugaDeficit(lut_path))

### BastankhahGaussianDeficit

The `BastankhahGaussianDeficit` model is implemented according to Bastankhah M and Porté-Agel F. "A new analytical model for wind-turbine wakes" J. Renew. Energy. 2014;70:116-23. The model is valid in the far wake only.

In [ ]:
from py_wake.deficit_models import BastankhahGaussianDeficit
plot_wake_deficit_map(BastankhahGaussianDeficit())

### IEA37SimpleBastankhahGaussianDeficit

The `IEA37SimpleBastankhahGaussian` model is implemented according to the [IEA task 37 documentation](https://github.com/byuflowlab/iea37-wflo-casestudies/blob/master/cs1-2/iea37-wakemodel.pdf) and is equivalent to BastankhahGaussian for $beta=1/\sqrt{8} \sim ct=0.9637188$. The model is valid in the far wake only.

In [ ]:
from py_wake.deficit_models import IEA37SimpleBastankhahGaussianDeficit
plot_wake_deficit_map(IEA37SimpleBastankhahGaussianDeficit())

### GCLDeficit

    Implemented according to:
            Larsen, G. C. (2009). A simple stationary semi-analytical wake model.
            Risoe National Laboratory for Sustainable Energy,
            Technical University of Denmark. Denmark.
            Forskningscenter Risoe. Risoe-R, No. 1713(EN)

    Description:
        based on an analytical solution of the thin shear layer approximation of the NS equations.
        The wake flow fields are assumed rotationally symmetric, and the rotor inflow fields
        are consistently assumed uniform.
        The effect of expansion is approximately accounted for by imposing suitable
        empirical downstream boundary conditions on the wake expansion that depend
        on the rotor thrust and the ambient turbulence conditions, respectively.

In [ ]:
from py_wake.deficit_models import GCLDeficit
plot_wake_deficit_map(GCLDeficit())

### NiayifarGaussianDeficit
  
    Implemented according to:
        Amin Niayifar and Fernando Porté-Agel
        Analytical Modeling of Wind Farms: A New Approach for Power Prediction
        Energies 2016, 9, 741; doi:10.3390/en9090741

    Features:
        - Wake expansion function of local turbulence intensity

    Description:
        The expansion rate 'k' varies linearly with local turbluence
        intensity: k = a1 I + a2. The default constants are set
        according to publications by Porte-Agel's group, which are based
        on LES simulations. Lidar field measurements by Fuertes et al. (2018)
        indicate that a = [0.35, 0.0] is also a valid selection.

In [ ]:
from py_wake.deficit_models import NiayifarGaussianDeficit
plot_wake_deficit_map(NiayifarGaussianDeficit(a=[0.38, 4e-3]))

### ZongGaussianDeficit
  
    Implemented according to:
        Haohua Zong and Fernando Porté-Agel
        A momentum-conserving wake superposition method for
        wind farm power prediction
        J. Fluid Mech. (2020), vol. 889, A8; doi:10.1017/jfm.2020.77

    Features:
        - Wake expansion function of local turbulence intensity
        - New wake width expression following the approach by
          Shapiro et al. (2018)

    Description:
        Extension of the Niayifar et al. (2016) implementation with Shapirio
        wake width expression, which uses the near-wake length estimation by
        Vermeulen (1980).

In [ ]:
from py_wake.deficit_models import ZongGaussianDeficit
plot_wake_deficit_map(ZongGaussianDeficit(a=[0.38, 4e-3]))

### GCLDeficit

    Implemented according to:
            Larsen, G. C. (2009). A simple stationary semi-analytical wake model. 
            Risø National Laboratory for Sustainable Energy, 
            Technical University of Denmark. Denmark. 
            Forskningscenter Risoe. Risoe-R, No. 1713(EN)

    Description:
        based on an analytical solution of the thin shear layer approximation of the NS equations. 
        The wake flow fields are assumed rotationally symmetric, and the rotor inflow fields 
        are consistently assumed uniform.
        The effect of expansion is approximately accounted for by imposing suitable 
        empirical downstream boundary conditions on the wake expansion that depend 
        on the rotor thrust and the ambient turbulence conditions, respectively. 

In [ ]:
from py_wake.deficit_models import GCLDeficit
plot_wake_deficit_map(GCLDeficit())

### Compare deficit models

In [ ]:
deficitModels = [NOJDeficit(),
                 FugaDeficit(lut_path),
                 BastankhahGaussianDeficit(),
                 IEA37SimpleBastankhahGaussianDeficit(),
                 NiayifarGaussianDeficit(),
                 ZongGaussianDeficit(),
                 GCLDeficit()]

**Deficit along center line**

In [ ]:
plt.figure(figsize=((10,6)))
for deficitModel in deficitModels:
    X, Y, deficit = _map(deficitModel.calc_deficit, xy=(np.linspace(-200,500,300), 0))
    plt.plot(X[0], deficit[0], label=deficitModel.__class__.__name__)
plt.title("Center line deficit")
plt.xlabel('x/D')
plt.ylabel('Deficit [m/s]')
plt.legend()

**Deficit profile 2D downstream**

In [ ]:
plt.figure(figsize=((10,6)))
for deficitModel in deficitModels:
    X, Y, deficit = _map(deficitModel.calc_deficit, xy=(2*D, np.linspace(-200,200,300)))
    plt.plot(Y[:,], deficit[:,0], label=deficitModel.__class__.__name__)

plt.legend()

### Implement your own deficit models
Deficit models must subclass `DeficitModel`and thus must implement the `calc_deficit` method and a class variable, `args4deficit` specifying the arguments required by its `calc_deficit` method

```python 
class DeficitModel(ABC):
    args4deficit = ['WS_ilk', 'dw_ijlk']

    @abstractmethod
    def calc_deficit(self):
        """Calculate wake deficit caused by the x'th most upstream wind turbines
        for all wind directions(l) and wind speeds(k) on a set of points(j)

        This method must be overridden by subclass

        Arguments required by this method must be added to the class list
        args4deficit

        See class documentation for examples and available arguments

        Returns
        -------
        deficit_ijlk : array_like
        """
```

In [ ]:
from py_wake.deficit_models import WakeDeficitModel
from numpy import newaxis as na
class MyDeficitModel(WakeDeficitModel):
    args4deficit = ['WS_ilk', 'dw_ijlk', 'cw_ijlk']

    def calc_deficit(self, WS_ilk, dw_ijlk, cw_ijlk,**_):
        # 10% deficit in downstream triangle
        ws_10pct_ijlk = 0.1*WS_ilk[:,na]
        triangle_ijlk = ((.2*dw_ijlk) >cw_ijlk)
        return ws_10pct_ijlk *triangle_ijlk

plot_wake_deficit_map(MyDeficitModel())


## Superposition models
The super position models calculates the effective wind speed given the local wind speed and deficits (typically from multiple sources)

### LinearSum

In [ ]:
linear_sum = IEA37SimpleBastankhahGaussian(site, windTurbines, superpositionModel=LinearSum())
plt.figure(figsize=(10,4))
linear_sum([0,200],[0,0],wd=270,ws=10).flow_map().plot_wake_map(levels=np.arange(1,10))

### SquaredSum

In [ ]:
from py_wake.superposition_models import SquaredSum
squared_sum = IEA37SimpleBastankhahGaussian(site, windTurbines, superpositionModel=SquaredSum())
plt.figure(figsize=(10,4))
squared_sum([0,200],[0,0],wd=270,ws=10).flow_map().plot_wake_map(levels=np.arange(1,10))

### MaxSum

In [ ]:
from py_wake.superposition_models import MaxSum
max_sum = IEA37SimpleBastankhahGaussian(site, windTurbines, superpositionModel=MaxSum())
plt.figure(figsize=(10,4))
max_sum([0,200],[0,0],wd=270,ws=10).flow_map().plot_wake_map(levels=np.arange(1,10))

## Blockage deficit models

The blockage deficit models compute the blockage effects caused by a single wind turbine. Their structure are quite similar to the [wake deficit models](#Wake-deficit-models). They model upstream blockage effects (wind speed reduction) and in addition, some models also models downstream speed-up effects.

### SelfSimilarityDeficit
Simple inductionmodel model, described in [N. Troldborg, A.R. Meyer Fortsing, Wind Energy, 2016](https://onlinelibrary.wiley.com/doi/full/10.1002/we.2137)

In [ ]:
from py_wake.deficit_models import SelfSimilarityDeficit
plot_blockage_deficit_map(SelfSimilarityDeficit())

### SelfSimilarityDeficit2020

This is an updated version of [N. Troldborg, A.R. Meyer Fortsing, Wind Energy, 2016](https://onlinelibrary.wiley.com/doi/full/10.1002/we.2137). The new features are found in the radial and axial functions:

1. Radially Eq. (13) is replaced by a linear fit, which ensures the induction half width, `r12`, to continue to diminish approaching the rotor. This avoids unphysically large lateral induction tails, which could negatively influence wind farm simulations.
2. The value of gamma in Eq. (8) is revisited. Now gamma is a function of CT and axial coordinate to force the axial induction to match the simulated results more closely. The fit is valid over a larger range of thrust coefficients and the results of the constantly loaded rotor are excluded in the fit.

In [ ]:
from py_wake.deficit_models import SelfSimilarityDeficit2020
plot_blockage_deficit_map(SelfSimilarityDeficit2020())

### FugaDeficit

The FugaDeficit model calculates the wake deficit based on a set op look-up tables computed by a linearized RANS solver. The look-up tables be created in advance using the [Fuga GUI](https://orbit.dtu.dk/en/publications/developments-of-the-offshore-wind-turbine-wake-model-fuga)

The fugaDeficit models both near wake, far wake and blockage deficit.

Note, the present look-up table generator introduces some unphysical wriggles in the blockage deficit/speed-up

In [ ]:
from py_wake.deficit_models import FugaDeficit
plot_blockage_deficit_map(FugaDeficit())

### VortexCylinder

Induced velocity from a semi-infinite cylinder of tangential vorticity, extending along the z axis.

This model is an adapted version of the one published by Emmanuel Branlard at https://github.com/ebranlard/wiz/blob/master/wiz/VortexCylinder.py

References:

- E. Branlard, M. Gaunaa, Cylindrical vortex wake model: right cylinder, Wind Energy, 2014, https://onlinelibrary.wiley.com/doi/full/10.1002/we.1800
- E. Branlard, Wind Turbine Aerodynamics and Vorticity Based Method, Springer, 2017
- E. Branlard, A. Meyer Forsting, Using a cylindrical vortex model to assess the induction zone in front of aligned and yawed rotors, in Proceedings of EWEA Offshore Conference, 2015, https://orbit.dtu.dk/en/publications/using-a-cylindrical-vortex-model-to-assess-the-induction-zone-inf

In [ ]:
from py_wake.deficit_models import VortexCylinder
plot_blockage_deficit_map(VortexCylinder())

### VortexDipole

The vorticity originating from a wind turbine can be represented by a vortex dipole line (see Appendix B in [2]). The induction estimated by such a representation is very similar to the results given by the more complex vortex cylinder model in the far-field r/R > 6 [1,2]. The implementation follows the relationships given in [1,2]. This model is an adapted version of the one published by Emmanuel Branlard: https://github.com/ebranlard/wiz/blob/master/wiz/VortexDoublet.py

References:
- [1] Emmanuel Branlard et al 2020 J. Phys.: Conf. Ser. 1618 062036
- [2] Branlard, E, Meyer Forsting, AR. Wind Energy. 2020; 23: 2068– 2086.  https://doi.org/10.1002/we.2546

In [ ]:
from py_wake.deficit_models import VortexDipole
plot_blockage_deficit_map(VortexDipole())

### RankineHalfBody

A simple induction model using a Rankine Half Body to represent the induction introduced by a wind turbine. The source strength is determined enforcing 1D momentum balance at the rotor disc.

References:

- B Gribben, G Hawkes - A potential flow model for wind turbine induction and wind farm blockage - Technical Paper, Frazer-Nash Consultancy, 2019

In [ ]:
from py_wake.deficit_models import RankineHalfBody
plot_blockage_deficit_map(RankineHalfBody())

### HybridInduction
The idea behind this model originates from [2,3], which advocates to combine near-rotor and farfield approximations of a rotor's induced velocities. Whereas in [1,2] the motivation is to reduce the computational effort, here the already very fast self-similar model [1] is combined with the vortex dipole approximation in the far-field, as the self-similar one is optimized for the near-field (r/R > 6, x/R < 1) and misses the acceleration around the wake for x/R > 0. The combination of both allows capturing the redistribution of energy by blockage. Location at which to switch from near-rotor to far-field can be altered though by setting switch_radius.

References:
1. N. Troldborg, A.R. Meyer Fortsing, Wind Energy, 2016
2. Emmanuel Branlard et al 2020 J. Phys.: Conf. Ser. 1618 062036
3. Branlard, E, Meyer Forsting, AR. Wind Energy. 2020; 23: 2068– 2086. https://doi.org/10.1002/we.2546

In [ ]:
from py_wake.deficit_models import HybridInduction
plot_blockage_deficit_map(HybridInduction())

### Rathmann

Ole Sten Rathmann (DTU) developed in 2020 an approximation to the vortex cylinder solution (E. Branlard and M. Gaunaa, 2014). In speed it is comparable to the vortex dipole method, whilst giving a flow-field nearly identical to the vortex cylinder model for x/R < -1. Its centreline deficit is identical to the vortex cylinder model, whilst using a radial shape function that depends on the opening of the vortex cylinder seen from a point upstream. To simulate the speed-up downstream the deficit is mirrored in the rotor plane with a sign change.

In [ ]:
from py_wake.deficit_models import Rathmann
plot_blockage_deficit_map(Rathmann())

### Compare blokage deficit models

In [ ]:
blockagedeficitModels = [SelfSimilarityDeficit(),
                 SelfSimilarityDeficit2020(),
                 FugaDeficit(),
                 VortexCylinder(),
                 VortexDipole(),
                 RankineHalfBody(),
                 HybridInduction(),
                 Rathmann()]

**Deficit along center line**

In [ ]:
plt.figure(figsize=((10,6)))
for deficitModel in blockagedeficitModels:
    X, Y, deficit = _map(deficitModel.calc_deficit, xy=(np.linspace(-200,100,300), 0))
    plt.plot(X[0], deficit[0], label=deficitModel.__class__.__name__)
plt.title("Center line deficit")
plt.xlabel('x/D')
plt.ylabel('Deficit [m/s]')
plt.legend()

**Deficit profile 1 up- and downstream**

In [ ]:
for d in [-1,1]:
    plt.figure(figsize=((10,6)))
    for deficitModel in blockagedeficitModels:
        X, Y, deficit = _map(deficitModel.calc_deficit, xy=(d*D, np.linspace(-200,200,300)))
        plt.plot(Y[:,], deficit[:,0], label=deficitModel.__class__.__name__)
    plt.title("%sD %sstream"%(abs(d),('down','up')[d<0]))
    plt.ylim([-.5,.5])
    plt.xlabel('y/D')
    plt.ylabel('Deficit [m/s]')
    plt.legend()

## Rotor-average models

In the plots below, it is clearly seen that the wind speed varies over the rotor, and that the the rotor-average wind speed is not well-defined by the wind sped at the rotor center.


In [ ]:
from py_wake.superposition_models import SquaredSum
from py_wake.flow_map import HorizontalGrid, YZGrid
R = D/2
wfm = IEA37SimpleBastankhahGaussian(site, windTurbines, superpositionModel=SquaredSum())
sim_res = wfm([0,200],[0,0],wd=270,ws=10)
fig,(ax1,ax2) = plt.subplots(1,2, figsize=(10,4))
ax1.set_xlabel("x [m]"), ax1.set_ylabel("y [m]")
sim_res.flow_map(HorizontalGrid(extend=.1)).plot_wake_map(10, ax=ax1, plot_colorbar=False)
sim_res.flow_map(YZGrid(x=200)).plot_wake_map(10, ax=ax2)
ax2.plot([-100,100],[70,70],'-.')
ax2.set_xlabel("y [m]"), ax1.set_ylabel("z [m]")

plt.figure()
flow_map = sim_res.flow_map(HorizontalGrid(x=[200], y=np.arange(-80, 80)))

for x in [-.5,.5]:
    plt.gca().axvline(x,color='grey')
plt.plot(flow_map.Y[:, 0]/D, flow_map.WS_eff_xylk[:, 0, 0, 0], '-.', label='Hub-height profile')
plt.plot([-.5,.5],[7.73,7.73],label='Rotor average: 7.73m/s')
rc_ws = flow_map.WS_eff_xylk[80, 0, 0, 0]
plt.plot(flow_map.Y[80, 0]/D, rc_ws,'.', ms=10, label='Rotor center: %.2fm/s'%rc_ws)
plt.legend()
plt.xlabel("y/D")
plt.ylabel('Wind speed [m/s]')

A better estimate of the rotor-average wind speed can be obtained by a (weighted) mean of the wind speed at a number of points covering the rotor. Here is a some model examples.

In [ ]:
from py_wake.rotor_avg_models import RotorCenter, GridRotorAvg, EqGridRotorAvg, GQGridRotorAvg, CGIRotorAvg, PolarGridRotorAvg,polar_gauss_quadrature
from py_wake.flow_map import HorizontalGrid
R=D/2
def plot_rotor_avg_model(rotorAvgModel, name):
    plt.figure()
    m = rotorAvgModel
    wfm = IEA37SimpleBastankhahGaussian(site,windTurbines,rotorAvgModel=m)
    ws_eff = wfm([0, 200], [0, 0], wd=270, ws=10).WS_eff_ilk[1,0,0]
    plt.title(name)
    c = plt.scatter(m.nodes_x, m.nodes_y,c=m.nodes_weight,label="%.2fm/s"%(ws_eff))
    plt.colorbar(c,label='weight')
    plt.gca().add_artist(plt.Circle((0,0),1,fill=False))
    plt.axis('equal')
    plt.xlabel("y/R [m]"); plt.ylabel('z/R [m]')
    plt.xlim([-1.5,1.5])
    plt.ylim([-1.5,1.5])
    plt.legend()

### RotorCenter
The `RotorCenter` model determines the rotor average wind speed from the rotor center point

In [ ]:
plot_rotor_avg_model(RotorCenter(), 'RotorCenter')

### GridRotorAvg

The `GridRotorAvg` defines a set of points in cartesian coordinates

In [ ]:
x = y = np.array([-.6, 0,.6])
plot_rotor_avg_model(GridRotorAvg(x,y,nodes_weight = [0.25,.5,.25]), 'Grid_4')

### EqGridRotorAvg

The `EqGridRotorAvg` defines a NxN equidistant cartesian grid of points and discards points outside the rotor

In [ ]:
plot_rotor_avg_model(EqGridRotorAvg(4), 'Grid_4')

### GQGridRotorAvg

The `GQGridRotorAvg` defines a grid of M x N cartesian grid points using Gaussian quadrature coordinates and weights

In [ ]:
plot_rotor_avg_model(GQGridRotorAvg(4,3), 'GQGrid_4,3')

### PolarGridRotorAvg

The `PolarGridRotorAvg` defines a grid in polar coordinates.

In [ ]:
theta = np.linspace(-np.pi,np.pi,6, endpoint=False)
r = 2/3
plot_rotor_avg_model(PolarGridRotorAvg(nodes_r =r, nodes_theta=theta, nodes_weight=None), 'PolarGrid_6')

The polar grid can be combined with Gaussian Quadrature. This is similar to the implementation in FusedWake (https://gitlab.windenergy.dtu.dk/TOPFARM/FUSED-Wake/-/blob/master/fusedwake/gcl/fortran/GCL.f)

In [ ]:
plot_rotor_avg_model(PolarGridRotorAvg(*polar_gauss_quadrature(4,10)), 'PolarGrid_4,10')

### CGIRotorAvg
Circular Gauss integration with 4,7,9 or 21 points as defined in Abramowitz M and Stegun A. Handbook of Mathematical Functions. Dover: New York, 1970.

In [ ]:
for n in [4,7,9,21]:
    plot_rotor_avg_model(CGIRotorAvg(n), 'CGIRotorAvg_%d'%n)

### Compare rotor-average models
In general, the compuational cost and the accuracy of the estimate increases with the number of points, but the distribution of the points also has an impact.

The plot below shows the absolute error of the estimated rotor-average wind speed for the wind directions 
270$\pm$30$^\circ$ (i.e. wind directions with more than 1% deficit) for a number of different rotor-average models

In [ ]:
grid_models = [EqGridRotorAvg(i) for i in range(1,10)]
wd_lst = np.arange(240,301)

def get_ws_eff(rotorAvgModel):
    wfm = IEA37SimpleBastankhahGaussian(site,windTurbines,rotorAvgModel=rotorAvgModel)
    return wfm([0, 200], [0, 0], wd=wd_lst, ws=10).WS_eff_ilk[1,:,0]

ws_ref = get_ws_eff(EqGridRotorAvg(200)) # Use 200x200 points (31700 inside the rotor) to determine the reference value

def get_n_err(rotorAvgModel):
    ws_mean_err = np.abs(get_ws_eff(rotorAvgModel) - ws_ref).mean()
    return len(rotorAvgModel.nodes_x), ws_mean_err
        

plt.gca().axhline(0, color='grey',ls='--')
plt.plot(*zip(*[get_n_err(m) for m in grid_models]), label='Grid_x')
model_lst = [('RotorCenter', EqGridRotorAvg(1)),
             ('Grid_4', EqGridRotorAvg(4)),             
             ('PolarGrid_4,10', PolarGridRotorAvg(*polar_gauss_quadrature(4,10))),
             ('GQGrid_4,3', GQGridRotorAvg(4, 3))] + \
            [('CGI_%d'%n, CGIRotorAvg(n)) for n in [4,7,9,21]]
for name, model in model_lst:
    n,err = get_n_err(model)
    plt.plot(n,err,'.',ms=10, label="%s (%.4f)"%(name,err))
plt.xlabel('Number of rotor-average points')
plt.ylabel('Mean abs error (270$\pm30^\circ$) [m/s]')
plt.legend()

## Deflection models
The deflection models calculate the deflection of the wake due to yaw-misalignment, sheared inflow etc. 
Note, this is one of the four effects of skew inflow that is handled in PyWake, see [here](https://topfarm.pages.windenergy.dtu.dk/PyWake/notebooks/YawMisalignment.html).
The deflection models take as input the downwind and crosswind distances between the source wind turbines and the destination wind turbines/sites and calculate a new set of deflected downwind and crosswind distances.

In [ ]:
def plot_deflection(deflectionModel):
    from py_wake import IEA37SimpleBastankhahGaussian
    from py_wake.examples.data.iea37._iea37 import IEA37Site, IEA37_WindTurbines

    site = IEA37Site(16)
    x, y = [0, 400, 800], [0, 0, 0]
    windTurbines = V80()
    wfm = IEA37SimpleBastankhahGaussian(site, windTurbines, deflectionModel=deflectionModel)

    yaw = [-20,20,0]

    plt.figure(figsize=(14,4))
    fm = wfm(x, y, yaw=yaw, wd=270, ws=10).flow_map()
    fm.plot_wake_map()
    fm.min_WS_eff().plot(color='k', ls='--')

### JimenezWakeDeflection

The `JimenezWakeDeflection` model is implemented according to Jiménez, Á., Crespo, A. and Migoya, E. (2010), Application of a LES technique to characterize the wake deflection of a wind turbine in yaw. Wind Energ., 13: 559-572. doi:10.1002/we.380

In [ ]:
from py_wake.deflection_models import JimenezWakeDeflection
plot_deflection(JimenezWakeDeflection())

### FugaDeflection

In [ ]:
from py_wake.deflection_models import FugaDeflection
plot_deflection(FugaDeflection())

### Implement your own deflection models

Deficit models must subclass `DeficitModel` and thus must implement the `calc_deflection` method and a class variable, `args4deflection` specifying the arguments required by its calc_deflection method

```python

class DeflectionModel(ABC):
    args4deflection = ["ct_ilk"]

    @abstractmethod
    def calc_deflection(self, dw_ijl, hcw_ijl, **kwargs):
        """Calculate deflection

        This method must be overridden by subclass

        Arguments required by this method must be added to the class list
        args4deflection

        See documentation of EngineeringWindFarmModel for a list of available input arguments

        Returns
        -------
        dw_ijlk : array_like
            downwind distance from source wind turbine(i) to destination wind turbine/site (j)
            for all wind direction (l) and wind speed (k)
        hcw_ijlk : array_like
            horizontal crosswind distance from source wind turbine(i) to destination wind turbine/site (j)
            for all wind direction (l) and wind speed (k)
        """
```

In [ ]:
from py_wake.deflection_models import DeflectionModel
from numpy import newaxis as na
class MyDeflectionModel(DeflectionModel):
    args4deficit = ['dw_ijlk', 'cw_ijlk']

    def calc_deflection(self, dw_ijl, hcw_ijl, **_):
        dw_ijlk = dw_ijl[..., na] # add extra wind speed dimension
        hcw_ijlk =hcw_ijl[..., na] + .1*dw_ijlk # deflect 1/10 of the downstream distance
        dh_ijlk =np.zeros_like(hcw_ijl[..., na]) # no vertical deflection
        return dw_ijlk, hcw_ijlk, dh_ijlk

iea_my_deflection = IEA37SimpleBastankhahGaussian(site, windTurbines, deflectionModel=MyDeflectionModel())

plt.figure(figsize=(10,4))
iea_my_deflection(x, y, wd=270, ws=10).flow_map().plot_wake_map()


## Turbulence models

In [ ]:
# methods to plot turbulence map, used below to visualize and compare the deficit models
   
def plot_turb_map(turbulenceModel):
    xy = np.linspace(-200,500,200), np.linspace(-200,200,200)
    X,Y,deficit = _map(turbulenceModel.calc_added_turbulence, xy)
    c = plt.contourf(X,Y,deficit, levels=100, cmap='Blues')
    plt.colorbar(c, label="Added turbulence intensity [-]")
    plt.plot([0,0],[-1/2,1/2],'k')
    plt.ylabel("Crosswind distance [y/D]")
    plt.xlabel("downwind distance [x/D]")

### STF2005TurbulenceModel

Steen Frandsen model implemented according to IEC61400-1, 2005 and weight according to Steen Frandsen's [thesis](https://orbit.dtu.dk/en/publications/turbulence-and-turbulence-generated-structural-loading-in-wind-tu)

In [ ]:
from py_wake.turbulence_models import STF2005TurbulenceModel
plot_turb_map(STF2005TurbulenceModel())

### STF2017TurbulenceModel

Steen Frandsen model implemented according to IEC61400-1, 2017 and weight according to Steen Frandsen's [thesis](https://orbit.dtu.dk/en/publications/turbulence-and-turbulence-generated-structural-loading-in-wind-tu)

In [ ]:
from py_wake.turbulence_models import STF2017TurbulenceModel
plot_turb_map(STF2017TurbulenceModel())

**STF20XXTurbulenceModel with IEC-based spread angle**

The `STF2005TurbulenceModel` and `STF2017TurbulenceModel` take a `weight_function` input which defaults to the bell-shaped `FrandsenWeight` defined in Steen Frandsen's thesis. As an alternative the `IECWeight` applies the full added turbulence in a 21.6$^\circ$ spread angle up to 10 diameter downstream. 

Note, this is a debatable interpretation of the IEC standard which includes a 6% contribution from neighbouring wind turbines when calculating the omni-directional effective turbulence intensity. These 6% maps to a spread angle of 360$^\circ\cdot$ 6% = 21.6$^\circ$.

Note, the IEC standard includes more concepts which is not implemented in PyWake

In [ ]:
from py_wake.turbulence_models import STF2017TurbulenceModel, IECWeight
from py_wake.superposition_models import SqrMaxSum
plot_turb_map(STF2017TurbulenceModel(addedTurbulenceSuperpositionModel=SqrMaxSum(), 
                                     weight_function=IECWeight(distance_limit=10)))

### GCLTurbulence

Gunner Chr. Larsen model implemented according to 
    
    Pierik, J. T. G., Dekker, J. W. M., Braam, H., Bulder, B. H., Winkelaar, D., Larsen, G. C., Morfiadakis, E., Chaviaropoulos, P., Derrick, A., & Molly, J. P. (1999). European wind turbine standards II (EWTS-II). In E. L. Petersen, P. Hjuler Jensen, K. Rave, P. Helm, & H. Ehmann (Eds.), Wind energy for the next millennium. Proceedings (pp. 568-571). James and James Science Publishers.

In [ ]:
from py_wake.turbulence_models import GCLTurbulence
plot_turb_map(GCLTurbulence())

### CrespoHernandez

    Implemented according to:
    A. Crespo and J. Hernández
    Turbulence characteristics in wind-turbine wakes
    J. of Wind Eng. and Industrial Aero. 61 (1996) 71-85

In [ ]:
from py_wake.turbulence_models import CrespoHernandez
plot_turb_map(CrespoHernandez())

### Compare Turbulence models

In [ ]:
turbulenceModels = [STF2005TurbulenceModel(),
                 STF2017TurbulenceModel(),
                 STF2017TurbulenceModel(addedTurbulenceSuperpositionModel=SqrMaxSum(), weight_function=IECWeight(10)),
                 GCLTurbulence(),
                 CrespoHernandez()]

**turbulence intensity along center line**

In [ ]:
plt.figure(figsize=((10,6)))
for turbulenceModel in turbulenceModels:
    X, Y, ti = _map(turbulenceModel.calc_added_turbulence, xy=(np.linspace(-200,500,300), 0))
    l = turbulenceModel.__class__.__name__
    if l.startswith('STF'):
        l+=f"({turbulenceModel.apply_weight.__self__.__class__.__name__})"
    plt.plot(X[0], ti[0], label=l)

plt.legend()

**Deficit profile 2D downstream**

In [ ]:
plt.figure(figsize=((10,6)))
for turbulenceModel in turbulenceModels:
    X, Y, ti = _map(turbulenceModel.calc_added_turbulence, xy=(2*D, np.linspace(-200,200,300)))
    plt.plot(Y[:,], ti[:,0], label=turbulenceModel.__class__.__name__)

plt.legend()

## Ground models

### Mirror
The Mirror ground model lets the ground mirror the wake deficit. It is implemented by adding wakes from underground mirrored wind turbines

In [ ]:
from py_wake.ground_models import Mirror
wfm = NOJ(site, windTurbines, k=.5, groundModel=Mirror())
wfm([0], [0], wd=0).flow_map(YZGrid(x=0, y=np.arange(-300, 100, 1) + .1, z=np.arange(-100, 200))).plot_wake_map()
